<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mbart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training MBART model

In [ ]:
# check GPU avaliable

!nvidia-smi

In [ ]:
# install package
# mount to drive if dataset in google drive

from google.colab import drive
drive.mount('/content/gdrive')

!pip install simpletransformers
!pip install torch
!pip install sacrebleu

In [ ]:
# config model

import logging
import torch
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# load dataset
train_df = pd.read_csv('/content/gdrive/MyDrive/train.csv').astype(str)
eval_df = pd.read_csv('/content/gdrive/MyDrive/eval.csv').astype(str)
train_df["prefix"] = ""
eval_df["prefix"] = ""

# config parameter
BATCH_SIZE = 4
EPOCH = 4
LEARNING_RATE = 4e-5
PROJECT_NAME = 'MBART Machine Translation'

model_args = Seq2SeqArgs()
model_args.train_batch_size = BATCH_SIZE
model_args.eval_batch_size = BATCH_SIZE
model_args.num_train_epochs = EPOCH
model_args.learning_rate = LEARNING_RATE
model_args.max_seq_length = 512
model_args.max_length = 100
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 300000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = PROJECT_NAME

# use GPU if available
device = torch.cuda.is_available()

model = Seq2SeqModel(encoder_decoder_type="mbart",
                     encoder_decoder_name="facebook/mbart-large-50",
                     args=model_args,
                     use_cuda=device)

# website for more model
# https://huggingface.co/models?sort=downloads

In [ ]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

# Evaluate the model
result = model.eval_model(eval_df)

In [ ]:
# load model

import logging
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = Seq2SeqArgs()
model_args.max_length = 100
model_args.length_penalty = 1
model_args.num_beams = 10

device = torch.cuda.is_available()

# path model
PATH = '/content/outputs'

model = Seq2SeqModel(encoder_decoder_type="mbart",
                     encoder_decoder_name=PATH,
                     args=model_args,
                     use_cuda=device)

In [ ]:
# calculate bleu score

import sacrebleu
import pandas as pd

# load test data
test = pd.read_csv("/content/test.csv").astype(str)
thai_truth = [test.loc[test["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = test.loc[test["prefix"] == "translate english to thai"]["input_text"].tolist()
english_truth = [test.loc[test["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = test.loc[test["prefix"] == "translate thai to english"]["input_text"].tolist()

# calculate BLEU score
english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)
thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

In [ ]:
# test prediction Thai to English

to_predict = ['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ',
              'และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก',
              'โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
              'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน',
              'และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน',
              'แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)',
              'ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
              'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์',
              'ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']

predictions = model.predict(to_predict)
for i, info in enumerate(predictions):
  print(to_predict[i] + '\n' + info + '\n')

In [ ]:
# test prediction English to Thai

to_predict2 = ['This engineering project develops a system for detecting. ',
              'and classifying plant disease from photographs of melon leaves using deep learning.',
              'The system uses NVIDIA Jetson Nano to process images from cameras in the greenhouse.',
              'To develop a deep learning model, the images of the melon leaves from the greenhouses were collected.',
              'and the melon leaf disease were labelled.',
              'Then use the dataset to train the object detector model.',
              'which will be able to detect disease in melon leaves.',
              'and when the system detects a disease, the caretaker will be notified of the issue through the LINE application.',
              'In the process of selecting a deep learning model to be deployed on the Jetson Nano.']

predictions2 = model.predict(to_predict2)
for i, info in enumerate(predictions2):
  print(to_predict2[i] + '\n' + info + '\n')